In [19]:
# # Check the perplexity of the question stems
#!pip install evaluate transformers
from evaluate import load

perplexity = load("perplexity", module_type="metric")
texts = [
    "If colony means an area under the rule of a king, then the word anticolonial means…",
    "If nation means a country, then the word internationally means…",
    "If crypt means hidden and graph stands for writing, then the word cryptographer means…",
    "If fatigue means being tired, then the word misfatigue means…"
]

results = perplexity.compute(predictions=texts, model_id='gpt2')

for i, text in enumerate(texts):
    print(f"Text {i+1}: \"{text}\"")
    print(f"  Perplexity: {round(results['perplexities'][i], 2)}")

print(f"\nMean Perplexity: {round(results['mean_perplexity'], 2)}")


  0%|          | 0/1 [00:00<?, ?it/s]

Text 1: "If colony means an area under the rule of a king, then the word anticolonial means…"
  Perplexity: 129.35
Text 2: "If nation means a country, then the word internationally means…"
  Perplexity: 162.03
Text 3: "If crypt means hidden and graph stands for writing, then the word cryptographer means…"
  Perplexity: 428.36
Text 4: "If fatigue means being tired, then the word misfatigue means…"
  Perplexity: 194.89

Mean Perplexity: 228.66


In [15]:
#!pip install wordfreq
from wordfreq import zipf_frequency

group_1 = ['unfair', 'intergovernmental', 'zeal', 'joyful', 'electrician']
group_2 = ['misapprehension', 'antidisestablishment','incontrovertible', 'transmogrification', 'hyperpolarization', 'preindeliberated']

def print_zipf_scores(words, group_name):
    print(f"\n{group_name}:")
    for word in words:
        score = zipf_frequency(word, 'en')
        print(f"{word}: {score}")

print_zipf_scores(group_1, "Group 1")
print_zipf_scores(group_2, "Group 2")



Group 1:
unfair: 4.17
intergovernmental: 3.1
zeal: 3.28
joyful: 3.4
electrician: 3.26

Group 2:
misapprehension: 2.06
antidisestablishment: 0.0
incontrovertible: 2.46
transmogrification: 1.6
hyperpolarization: 1.69
preindeliberated: 0.0


In [18]:
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def evaluate_with_distractors(keyword: str, options: dict):
    keyword_embedding = model.encode(keyword, convert_to_tensor=True)
    results = {}

    for label, text in options.items():
        option_embedding = model.encode(text, convert_to_tensor=True)
        similarity = float(util.cos_sim(keyword_embedding, option_embedding))
        results[label] = similarity

    return results


keyword1 = "cryptographer"
options1 = {
    "Correct_Answer": "someone who studies secret writing",
    "Distractor_1": "someone who writes in secret",
    "Distractor_2": "someone who hides secret writing"
}
sim_scores1 = evaluate_with_distractors(keyword1, options1)
print("Cryptographer:", sim_scores1)


keyword2 = "intergovernmental"
options2 = {
    "Distractor_1": "against groups of political officials",
    "Correct_Answer": "between groups of political officials",
    "Distractor_2": "between kings and princes"
}
sim_scores2 = evaluate_with_distractors(keyword2, options2)
print("Intergovernmental:", sim_scores2)


Cryptographer: {'Correct_Answe': 0.4772760272026062, 'Distractor_1': 0.4447832703590393, 'Distractor_2': 0.43788760900497437}
Intergovernmental: {'Distractor_1': 0.4245999753475189, 'Correct_Answer': 0.5257800817489624, 'Distractor_2': 0.22636467218399048}
